In [ ]:
import os,sys
import kfp
from kubernetes import client as k8s_client, config

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username = os.getenv("USERNAME")
client = kfp.Client(existing_token=token)
config.load_incluster_config()
v1 = client.CoreV1Api()

In [ ]:
secret = client.V1Secret()
secret.metadata = client.V1ObjectMeta(name="mnist-secret")
secret.type = "Opaque"
secret.data = {
  "AWS_ACCESS_KEY_ID": "test",
  "AWS_SECRET_ACCESS_KEY": "test"
  }

v1.create_namespaced_secret(username, secret)

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(token, program='training'):

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="dkube-examples", run_script="python mnist/train.py",
                                    datasets='["mnist"]', outputs='["mnist"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]')

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
                                    transformer_image='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    transformer_project="dkube-examples",
                                    transformer_code='mnist/transformer.py').after(train)
    

In [ ]:
client.create_run_from_pipeline_func(mnist_pipeline, arguments={"token":token})

In [ ]:
#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline.zip")
client.upload_pipeline("mnist-pipeline.zip")